In [20]:
import cv2
import numpy as np
import os
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.model_selection import train_test_split
import copy
from PIL import Image
from collections import defaultdict
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader,Dataset,random_split
from sklearn.metrics import accuracy_score
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_mcs_mouth.xml')

[ERROR:0@610.031] global persistence.cpp:566 open Can't open file: '/usr/local/lib/python3.10/site-packages/cv2/data/haarcascade_mcs_mouth.xml' in read mode


In [3]:
def extract_data(path,create_test=None):
    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]

    file=os.listdir(path)

    for entity in file:
        try:
            entityPath=os.listdir(f"{path}/{entity}")
            for i,data in enumerate(entityPath):
                try:
                    current_img=cv2.imread(f"{path}/{entity}/{data}")
                    if current_img is not None:
                        if (create_test is not None and ((len(entityPath)>1) and (i==0))):
                            X_test.append(current_img)
                            y_test.append(entity)
                        else:
                            X_train.append(current_img)
                            y_train.append(entity)
                    else:
                        print(f"{path}/{entity}/{data} is None type")
                except Exception as e:
                    print(f"Erreur avec {path}/{entity}/{data} : {e}")
        except Exception as ex:
            print(f"Erreur avec {path}/{entity} : {ex}")
    
    print(f"Number of training sample : {len(X_train)}\n")
    if create_test is not None:
        print(f"Number of test sample : {len(X_test)}\n")
        return X_train,y_train,X_test,y_test
    else:
        return X_train,y_train

In [4]:
def extract_face_and_eyes(img):
    if not isinstance(img,np.ndarray):
        img=cv2.imread(img)

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(60,60)
    )

    if len(faces)==0:
        return [],[]
    else:
        trueFaces=[]
        trueEyes=[]
        for i,face in enumerate(faces):
            x,y,w,h=face
            imgFace=img[y:y+h,x:x+w]
            eyes=eye_cascade.detectMultiScale(imgFace)
            if len(eyes)>=2:
                trueFaces.append(face)
                if eyes[0][0]<eyes[1][0]:
                    x2=eyes[0][0]
                    w2=eyes[1][0]+eyes[1][2]-x2
                else:
                    x2=eyes[1][0]
                    w2=eyes[0][0]+eyes[0][2]-x2
                if eyes[0][1]<eyes[1][1]:
                    y2=eyes[0][1]
                    h2=eyes[1][1]+eyes[1][3]-y2
                else:
                    y2=eyes[1][1]
                    h2=eyes[0][1]+eyes[0][3]-y2
                trueEyes.append((x2+x,y2+y,w2,h2))
        return trueFaces,trueEyes
    

In [5]:
def extract_face(img):
    if not isinstance(img,np.ndarray):
        img=cv2.imread(img) #si on passe un path ça read l'img

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(60,60)
    )

    if len(faces)==0:
        return img #retourne l'img si aucun visage n'est detecté
    
    #si je veux detecter tous les visages présents je dois faire une boucle for x,y,w,h in faces: et return une liste d'img crops
    x,y,w,h=faces[0] #faces retourne n tuples (x,y,h,w) correspondants aux coords du visage
    face=img[y:y+h,x:x+w] #retourne l'img crop aux coords detectées
    return face

In [53]:
X_train,y_train=extract_data("nous/Train")
X_faces=[]
for img in X_train:
    X_faces.append(extract_face(img))

labels={label:i for i,label in enumerate(sorted(set(y_train)))}
#X_train,X_test,y_train,y_test=train_test_split(X_faces,y_train,test_size=0.2,random_state=1,stratify=y_train)

Invalid SOS parameters for sequential JPEG


Number of training sample : 20



In [62]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # Moyennes ImageNet
                         [0.229, 0.224, 0.225])  # Écarts-types ImageNet
])

In [55]:
class FaceDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]

        # Conversion image
        image = Image.fromarray(image.astype(np.uint8))
        if self.transform:
            image = self.transform(image)

        # 🔁 Convertir label en tensor (si ce n'est pas déjà le cas)
        label = torch.tensor(label, dtype=torch.long)

        return image, label

In [ ]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)  # → array d'entiers
dataset = FaceDataset(X_faces, y_train_encoded, transform=transform)

In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)

# Geler les couches convolutionnelles
for param in model.parameters():
    param.requires_grad = False

# Adapter la dernière couche (classification)
num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

In [61]:
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)  # On n'entraîne que la fc

# Boucle d'entraînement
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.7389
Epoch 2, Loss: 0.7173
Epoch 3, Loss: 0.6696
Epoch 4, Loss: 0.5453
Epoch 5, Loss: 0.4666


In [66]:
X_test,y_test=extract_data("CameraFaces")
X_facesTest=[]
for img in X_test:
    X_facesTest.append(extract_face(img))

labels={label:i for i,label in enumerate(sorted(set(y_test)))}
le = LabelEncoder()
y_test_enc = le.fit_transform(y_test)  # → array d'entiers
datasetTest = FaceDataset(X_facesTest, y_test_enc, transform=transform)
test_loader = DataLoader(datasetTest, batch_size=32, shuffle=True)

Number of training sample : 6



In [67]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, 1) 

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
print(f"Validation Accuracy: {acc:.4f}")

Validation Accuracy: 1.0000


In [65]:
torch.save(model.state_dict(), "resNet50V1.pth")